# PyCity Schools Analysis

* An additional metric was calculated, that checks how many students have not just passed on average but who have passed both their reading and their math scores.  This ensures that we are counting the students that may have only passed reading OR math as not passing.  I have included this metric on the far right of the tables as "% Passing Both"

* The size of the school appears to a very important metric determining the success of the students.  The change in student success happens at schools between 2000-3000 students.  In small schools, more than 90% of the students are passing both math and reading, but a midsize school has significantly lower reading and math scores.  Meanwhile, both Large and Extra Large schools are performing equally poorly with approximately 80% of students passing reading and only 66% of students passing math! However, an extra small school does not seem to correspond to a continued increase in scores.  They perform equally with the standard small schools.

* Surprisingly, increasing School Budget per student does not result in higher test scores or increased passing rates.  The 6 schools with lowest budget have the highest average test scores and passing rates.  Is this because larger schools are somehow more expensive to operate?

* All of the top performing schools are Charter Schools and the bottom performing schools are all District Schools. 

* Each school has consistent test scores across the grades.
---

### PyCity School Data
* The analysed data comes from the PyCity school District.  We have two datasets.  One that describes the 15 schools in the district and the other dataset describes all the students at those schools and their test scores in reading and math.

In [180]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete = school_data_complete.rename(columns={"school_name":"Schools"})
school_data_complete.head()

,Student ID,student_name,gender,grade,Schools,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [181]:
# counting and calculating the requested variables
total_schools = school_data["school_name"].count()
total_students = student_data["Student ID"].count()
total_budget = school_data["budget"].sum()
ave_math_score = student_data["math_score"].mean()
ave_read_score = student_data["reading_score"].mean()

# Here the code returns dataframes with just the kids passing math or reading
# so we can count them up and calculate percentages
pass_math_df = school_data_complete.loc[school_data_complete["math_score"] >= 70,:]
pass_read_df = school_data_complete.loc[school_data_complete["reading_score"] >= 70,:]

percent_pass_math = 100 * pass_math_df["Student ID"].count() / total_students
percent_pass_read = 100 * pass_read_df["Student ID"].count() / total_students

# the asked for percent overall passing calculated as the average of the math and read scores:
#I'll call this Overall Score:
# (ave_math_score + ave_read_score)/2
overall_score = (ave_math_score + ave_read_score)/2

# Here I'm checking which kids have passed BOTH Math AND Reading 
# which seems like a more useful percentage then the average combined score asked for in this exercise
pass_both = school_data_complete.loc[(school_data_complete["reading_score"] >= 70) & 
                                     (school_data_complete["math_score"] >= 70),:]
percent_pass_both = 100 * pass_both["Student ID"].count() / total_students

# Form the new dataframe with new titles
district_summary = pd.DataFrame({"Total Schools": [total_schools], "Total Students": [total_students], 
                                 "Total Budget": [total_budget], "Average Math Score": [ave_math_score], 
                                 "Average Reading Score": [ave_read_score], "% Passing Math": [percent_pass_math], 
                                 "% Passing Reading": [percent_pass_read], "% Passing Overall": overall_score, 
                                 "% Passing Both":[percent_pass_both]})
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,}".format)
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Average Math Score"] = district_summary["Average Math Score"].map("{:.4}".format)
district_summary["Average Reading Score"] = district_summary["Average Reading Score"].map("{:.4}".format)
district_summary["% Passing Reading"] = district_summary["% Passing Reading"].map("{:.4}".format)
district_summary["% Passing Math"] = district_summary["% Passing Math"].map("{:.4}".format)
district_summary["% Passing Both"] = district_summary["% Passing Both"].map("{:.4}".format)
district_summary["% Passing Overall"] = district_summary["% Passing Overall"].map("{:.4}".format)
district_summary


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall,% Passing Both
0,15,"39,170","$24,649,428",78.99,81.88,74.98,85.81,80.43,65.17


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

In [182]:
# group the complete dataframe by schools
school_groups = school_data_complete.groupby(['Schools', 'type'])
school_group_df = pd.DataFrame(school_groups["size","budget","math_score", "reading_score"].mean())
school_group_df["Budget per Student"] = school_group_df["budget"] / school_group_df["size"]

# count up how many students passed math and reading and both for each school using groupby 
# on the dataframe made for the district for only passing students, 
# the num variable is a series to be turned into a percentage and added to the final dataframe
school_pass_math = pass_math.groupby(["Schools"])
school_pass_math_num = school_pass_math["Student ID"].count()
school_pass_read = pass_read.groupby(["Schools"])
school_pass_read_num = school_pass_read["Student ID"].count()
##school_pass_overall = pass_overall.groupby(["Schools"])
##school_pass_overall_num = school_pass_overall["Student ID"].count()

# I'm adding this variable school_pass_both in addition to the work asked for because it 
# makes more sense then the variable asked for: average of reading and math
school_pass_both = pass_both.groupby(["Schools"])
school_pass_both_num = school_pass_both["Student ID"].count()


# now calculate the percentages:
school_group_df["% Passing Math"] = 100 * (school_pass_math_num/school_group_df["size"])
school_group_df["% Passing Reading"] = 100 * (school_pass_read_num/school_group_df["size"])
school_group_df["% Passing Overall"] = (school_group_df["% Passing Math"] + school_group_df["% Passing Reading"])/2

##school_group_df["% Passing Overall"] = 100 * (school_pass_overall_num/school_group_df["size"])

#below is also a better way to calculate the not very helpful overall passing rate
school_group_df["% Passing Both"] = 100 * (school_pass_both_num/school_group_df["size"])


# clean up the column names for clarity
# reset the type index into a column
school_group_df = school_group_df.reset_index(level=['type'])
school_group_df = school_group_df.rename(columns={"type": "School Type",
                                          "size":"Total Students",
                                          "budget":"Total School Budget",
                                          "math_score": "Average Math Score",
                                          "reading_score": "Average Reading Score"})

# reorganize the columns, This dataframe will also be used towards the end of this project:
school_group_df = school_group_df[["School Type", "Total Students", "Total School Budget", "Budget per Student", 
                                   "Average Math Score", "Average Reading Score", "% Passing Math", 
                                    "% Passing Reading", "% Passing Overall", "% Passing Both"]]

school_group_df


,School Type,Total Students,Total School Budget,Budget per Student,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall,% Passing Both
Schools,,,,,,,,,,
Bailey High School,District,4976.0,3124928.0,628.0,77.048432,81.033963,66.680064,81.933280,74.306672,54.642283
Cabrera High School,Charter,1858.0,1081356.0,582.0,83.061895,83.975780,94.133477,97.039828,95.586652,91.334769
Figueroa High School,District,2949.0,1884411.0,639.0,76.711767,81.158020,65.988471,80.739234,73.363852,53.204476
Ford High School,District,2739.0,1763916.0,644.0,77.102592,80.746258,68.309602,79.299014,73.804308,54.289887
Griffin High School,Charter,1468.0,917500.0,625.0,83.351499,83.816757,93.392371,97.138965,95.265668,90.599455
Hernandez High School,District,4635.0,3022020.0,652.0,77.289752,80.934412,66.752967,80.862999,73.807983,53.527508
Holden High School,Charter,427.0,248087.0,581.0,83.803279,83.814988,92.505855,96.252927,94.379391,89.227166
Huang High School,District,2917.0,1910635.0,655.0,76.629414,81.182722,65.683922,81.316421,73.500171,53.513884
Johnson High School,District,4761.0,3094650.0,650.0,77.072464,80.966394,66.057551,81.222432,73.639992,53.539172


## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [183]:
# sort the pretty dataframe based on the overall score
pretty_school_df = school_group_df.sort_values(by=['% Passing Overall'], ascending=False)

# format the columns to make them easier to read
pretty_school_df["Total Students"] = pretty_school_df["Total Students"].map(int)
pretty_school_df["Total School Budget"] = pretty_school_df["Total School Budget"].map(int)
pretty_school_df["Budget per Student"] = pretty_school_df["Budget per Student"].map(int)
pretty_school_df["Total School Budget"] = pretty_school_df["Total School Budget"].map("${}".format)
pretty_school_df["Budget per Student"] = pretty_school_df["Budget per Student"].map("${}".format)
pretty_school_df.head()

,School Type,Total Students,Total School Budget,Budget per Student,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall,% Passing Both
Schools,,,,,,,,,,
Cabrera High School,Charter,1858,$1081356,$582,83.061895,83.975780,94.133477,97.039828,95.586652,91.334769
Thomas High School,Charter,1635,$1043130,$638,83.418349,83.848930,93.272171,97.308869,95.290520,90.948012
Pena High School,Charter,962,$585858,$609,83.839917,84.044699,94.594595,95.945946,95.270270,90.540541
Griffin High School,Charter,1468,$917500,$625,83.351499,83.816757,93.392371,97.138965,95.265668,90.599455
Wilson High School,Charter,2283,$1319574,$578,83.274201,83.989488,93.867718,96.539641,95.203679,90.582567


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [184]:
# resort by the worst performing schools using the overall number
pretty_school_df = school_group_df.sort_values(by=['% Passing Overall'], ascending=True)
# format the columns to make them easier to read
pretty_school_df["Total Students"] = pretty_school_df["Total Students"].map(int)
pretty_school_df["Total School Budget"] = pretty_school_df["Total School Budget"].map(int)
pretty_school_df["Budget per Student"] = pretty_school_df["Budget per Student"].map(int)
pretty_school_df["Total School Budget"] = pretty_school_df["Total School Budget"].map("${}".format)
pretty_school_df["Budget per Student"] = pretty_school_df["Budget per Student"].map("${}".format)
pretty_school_df.head(5)

,School Type,Total Students,Total School Budget,Budget per Student,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall,% Passing Both
Schools,,,,,,,,,,
Rodriguez High School,District,3999,$2547363,$637,76.842711,80.744686,66.366592,80.220055,73.293323,52.988247
Figueroa High School,District,2949,$1884411,$639,76.711767,81.158020,65.988471,80.739234,73.363852,53.204476
Huang High School,District,2917,$1910635,$655,76.629414,81.182722,65.683922,81.316421,73.500171,53.513884
Johnson High School,District,4761,$3094650,$650,77.072464,80.966394,66.057551,81.222432,73.639992,53.539172
Ford High School,District,2739,$1763916,$644,77.102592,80.746258,68.309602,79.299014,73.804308,54.289887


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [185]:
# make 4 new dataframes with only 9,10,11,or 12th grade students using loc and a conditional
# use groupby and mean to get the average math/reading scores for each school
# then concatenate the series together to make the final dataframe for displaying either math or reading

grade09_df = school_data_complete.loc[school_data_complete["grade"] == "9th"]
grade10_df = school_data_complete.loc[school_data_complete["grade"] == "10th"]
grade11_df = school_data_complete.loc[school_data_complete["grade"] == "11th"]
grade12_df = school_data_complete.loc[school_data_complete["grade"] == "12th"]

ave_grade09 = grade09_df[["Schools","math_score","reading_score"]].groupby("Schools").mean()
ave_grade10 = grade10_df[["Schools","math_score","reading_score"]].groupby("Schools").mean()
ave_grade11 = grade11_df[["Schools","math_score","reading_score"]].groupby("Schools").mean()
ave_grade12 = grade12_df[["Schools","math_score","reading_score"]].groupby("Schools").mean()

# rename so its easier to merge into one table
ave_grade09 = ave_grade09.rename(columns={"math_score": "9th Average Math", "reading_score": "9th Average Reading"})
ave_grade10 = ave_grade10.rename(columns={"math_score": "10th Average Math", "reading_score": "10th Average Reading"})
ave_grade11 = ave_grade11.rename(columns={"math_score": "11th Average Math", "reading_score": "11th Average Reading"})
ave_grade12 = ave_grade12.rename(columns={"math_score": "12th Average Math", "reading_score": "12th Average Reading"})

# Put all the grade level scores together in one table
ave_grade_school = pd.merge(ave_grade09, ave_grade10, on="Schools")
ave_grade_school = pd.merge(ave_grade_school, ave_grade11, on="Schools")
ave_grade_school = pd.merge(ave_grade_school, ave_grade12, on="Schools")

# put the math scores together in one table
ave_grade_math = ave_grade_school[["9th Average Math", "10th Average Math", "11th Average Math", "12th Average Math"]]
ave_grade_math


,9th Average Math,10th Average Math,11th Average Math,12th Average Math
Schools,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [186]:
# put the reading scores together in one table
ave_grade_read = ave_grade_school[["9th Average Reading", "10th Average Reading", 
                                   "11th Average Reading", "12th Average Reading"]]
ave_grade_read

,9th Average Reading,10th Average Reading,11th Average Reading,12th Average Reading
Schools,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [187]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 595, 615, 635, 655]
group_names = ["<$595", "$595-615", "$615-635", "$635-655"]

In [188]:
school_group_df["Spending Range per Student"] = pd.cut(school_group_df["Budget per Student"], 
                                                       spending_bins, labels=group_names)
school_spending = school_group_df[["Average Math Score", "Average Reading Score", "% Passing Math", 
                                  "% Passing Reading", "% Passing Overall", "% Passing Both", 
                                  "Spending Range per Student"]].groupby("Spending Range per Student")
school_spend_df = school_spending.mean()
school_spend_df["# of Schools"] = school_spending.size()
school_spend_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall,% Passing Both,# of Schools
Spending Range per Student,,,,,,,
<$595,83.455399,83.933814,93.460096,96.610877,95.035486,90.369459,4
$595-615,83.599686,83.885211,94.230858,95.900287,95.065572,90.216324,2
$615-635,80.199966,82.425360,80.036217,89.536122,84.786170,72.620869,2
$635-655,77.866721,81.368774,70.347325,82.995575,76.671450,58.858741,7


## Scores by School Size

* Perform the same operations as above, based on school size.

In [193]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 3000, 4000, 5000]
group_names = ["Extra Small (<1000)", "Small (<2000)", "Medium (2000-3000)", "Large (3000-4000)", "Extra Large (4000-5000)"]

In [194]:
school_group_df["Size Range"] = pd.cut(school_group_df["Total Students"], 
                                                       size_bins, labels=group_names)
school_size = school_group_df[["Average Math Score", "Average Reading Score", "% Passing Math", 
                                  "% Passing Reading", "% Passing Overall", "% Passing Both", 
                                  "Size Range"]].groupby("Size Range")
school_size_df = school_size.mean()
school_size_df["# of Schools"] = school_size.size()
school_size_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall,% Passing Both,# of Schools
Size Range,,,,,,,
Extra Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831,89.883853,2
Small (<2000),83.374684,83.864438,93.599695,96.790680,95.195187,90.621535,5
Medium (2000-3000),78.429493,81.769122,73.462428,84.473577,78.968003,62.897703,4
Large (3000-4000),76.842711,80.744686,66.366592,80.220055,73.293323,52.988247,1
Extra Large (4000-5000),77.136883,80.978256,66.496861,81.339570,73.918215,53.902988,3


## Scores by School Type

* Perform the same operations as above, based on school type.

In [191]:
school_type = school_group_df[["Average Math Score", "Average Reading Score", "% Passing Math", 
                                  "% Passing Reading", "% Passing Overall", "% Passing Both", 
                                  "School Type"]].groupby("School Type")
school_type_df = school_type.mean()
school_type_df["# of Schools"] = school_type.size()
school_type_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall,% Passing Both,# of Schools
School Type,,,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660,90.432244,8
District,76.956733,80.966636,66.548453,80.799062,73.673757,53.672208,7
